# Imports

In [1]:
import sys
import os

sys.path.insert(0, "/".join(os.getcwd().split("/")[:-1]))
__package__ = "LoadModels_NoteBooks"


In [2]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from utils.global_functions import arg_parse, Metrics, MySampler, NewCrossEntropyLoss
from transformers import AutoModel
from  SingleModels.models.whisper import WhisperForEmotionClassification
from  SingleModels.whisper_nn import prepare_dataloader
import wandb
from utils.trainer import Trainer
from utils.data_loaders import BertDataset
import numpy as np

/home/zeerak.talat/.conda/envs/featuregen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load in Models/Path

In [3]:
args = {}
args["dropout"] =  0
args["output_dim"] =  7
args["dataset"] =  "meld"
args["learn_PosEmbeddings"] =  False
args["num_layers"] =  12

whisper = WhisperForEmotionClassification(args)
path = "WhisperStart/qgl7153h/toasty-sweep-9/best.pt"

In [4]:
whisper_state_dict = torch.load(f"../../TAV_Train/{path}" , map_location=torch.device('cpu'))['model_state_dict']
whisper.load_state_dict(whisper_state_dict)

<All keys matched successfully>

### Below is for encoder only load

In [22]:
whisper_state_dict = torch.load(f"../../TAV_Train/{path}" , map_location=torch.device('cpu'))['model_state_dict']
roberta_state_dict = whisper.state_dict()

for key in roberta_state_dict.keys():
    bert_key = 'bert.' + key  # prepend 'bert.' to the key
    if bert_key in bert_state_dict:
        # print(f"Loading {bert_key}"  , flush = True)
        roberta_state_dict[key] = bert_state_dict[bert_key]

# Check load properly

In [25]:
video_state_dict['videomae.encoder.layer.23.output.dense.bias']

tensor([-0.0754, -0.0241, -0.0955,  ...,  0.0763, -0.1577,  0.0778])

In [27]:
(roberta_state_dict['encoder.layer.23.output.dense.bias'] == bert_state_dict['videomae.encoder.layer.23.output.dense.bias']).all()

tensor(True)

# Try Test Set

In [5]:
whisper.eval()

WhisperForEmotionClassification(
  (whisper): WhisperForAudioClassification(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_feat

In [12]:
whisper.whisper.encoder

WhisperEncoder(
  (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
  (embed_positions): Embedding(1500, 1024)
  (layers): ModuleList(
    (0-23): 24 x WhisperEncoderLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((1024,), eps=

In [6]:
name = "../data/meld.pkl"
df = pd.read_pickle(f"{name}")
number_index = "emotion"
label_index = "emotion_label"
label2id = (
    df.drop_duplicates(label_index).set_index(label_index).to_dict()[number_index]
)
id2label = {v: k for k, v in label2id.items()}
id2label

{2: 'surprise',
 3: 'anger',
 0: 'neutral',
 1: 'joy',
 4: 'sadness',
 6: 'fear',
 5: 'disgust'}

In [7]:
df_test = prepare_dataloader(
        df[df['split'] == "test"], name, 1, "emotion", 2 , check="test"
    )
Metric = Metrics(num_classes=len(id2label), id2label=id2label, rank="cpu")

Are we running on mustard? False


In [8]:
trainer = Trainer(big_batch=23 , num_steps=1)
trainer.evaluate(whisper, df_test, Metric)

test:   0%|          | 0/2608 [00:00<?, ?it/s]2023-11-26 06:38:12.395510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 06:38:13.815661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
test: 100%|██████████| 2608/2608 [1:31:58<00:00,  2.12s/it]


 in test 
 Confusion Matrix = 
tensor([[651, 238,  52, 232,  53,  11,  19],
        [108, 189,  20,  58,  12,  10,   5],
        [ 63,  56,  87,  52,  10,   6,   7],
        [ 66,  46,  19, 192,  11,   5,   5],
        [ 52,  36,  12,  53,  46,   4,   5],
        [  9,  13,   4,  28,   6,   4,   3],
        [ 15,   8,   0,  16,   6,   2,   3]]) 


 in test 
 Acc = 
0.44938650727272034 


 in test 
 Prec = 
0.3184402287006378 


 in test 
 Rec = 
0.31386637687683105 


 in test 
 F1Weighted = 
0.45675623416900635 


 in test 
 F1Macro = 
0.30369332432746887 



Error: You must call wandb.init() before wandb.log()

 in test 
 Acc = 
0.44938650727272034 


 in test 
 Prec = 
0.3184402287006378 


 in test 
 Rec = 
0.31386637687683105 


 in test 
 F1Weighted = 
0.45675623416900635 


 in test 
 F1Macro = 
0.30369332432746887 


In [9]:
from sklearn.metrics import f1_score
import numpy as np

# Your confusion matrix
cm = np.array([[651, 238,  52, 232,  53,  11,  19],
        [108, 189,  20,  58,  12,  10,   5],
        [ 63,  56,  87,  52,  10,   6,   7],
        [ 66,  46,  19, 192,  11,   5,   5],
        [ 52,  36,  12,  53,  46,   4,   5],
        [  9,  13,   4,  28,   6,   4,   3],
        [ 15,   8,   0,  16,   6,   2,   3]])

# Convert the confusion matrix into actual and predicted labels
y_true = np.repeat(np.arange(cm.shape[0]), np.sum(cm, axis=1))
y_pred = np.repeat(np.arange(cm.shape[1]), np.sum(cm, axis=0))

# Calculate weighted F1 score
f1 = f1_score(y_true, y_pred, average='macro')

print(f"The weighted F1 score is {f1}")


The weighted F1 score is 0.6964451231409771


# Try Loading in out model, but index out encoder

In [18]:
import torch

x1 = torch.Tensor([2])
x2 = torch.Tensor([4])
x3 = torch.Tensor([3])

torch.mean(torch.stack([x1 , x2 , x3]))

tensor(3.)